IPython notebook created by BIOL3014/BINF7000 course staff. Material is derived from 2014-? Practical 2.

Bug fixes: m.stantoncook@uq.edu.au, m.boden@uq.edu.au

Source: https://github.com/UQ-BIOL3014/Practical2

---

# BIOL3014/BINF700 Practical 2

## Probabilistic motif discovery
---

* **Due:** 11AM 12/08/2015
* **Revision:** 1
* **Marks:** 
    * **BIOL3014** - 8 marks. 
    * **BINF7000** - 12 marks.
---


### Objectives 

Below are a number of exercises that aim to guide you through issues and help you understand concepts related primarily to:
* Motif discovery
* Gene ontology
* Statistical testing (BINF7000)

---


### Submission requirements

Please export this IPython notebook (with written answers & completed code) to `STUDENTNUMBER_P2.ipynb` notebook and submit it via the BlackBoard portal.

----


### Resources

#### Python resources:
* The UQ Bioinformatics Python Guide (on Blackboard)
* The [python 2 documentation]. For those unfamiliar with Python the [official tutorial] is recommended
* The software carpentry [novice python lessons]

[python 2 documentation]: https://docs.python.org/2/
[offical tutorial]: https://docs.python.org/2/tutorial/index.html
[novice python lessons]: http://swcarpentry.github.io/python-novice-inflammation/

#### Other:
* [Markdown cheatsheet]

[Markdown cheatsheet]: https://github.com/adam-p/markdown-here/wiki/Markdown-Here-Cheatsheet#links

---

## Part 1 (questions 1-4, 8 marks) ##
The target workload for Part 1 corresponds roughly to two hours of practical work. This work should ideally be completed within the next scheduled computer lab session (supported by a tutor or lecturer), and two-three hours of additional work that should be completed during unscheduled time. Part 2 and 3 are for BINF7000 students only.

We will be using the custom `uqbinfpy` Python modules for this practical. The `uqbinfpy` library is available at [GitHub]. Data files are available on Blackboard under the "Practicals/workshops" tab, as well as at [GitHub].

[GitHub]: https://github.com/UQ-BIOL3014/

You should have installed the uqbinfpy library during Practical 1.

Import the following modules into Python:


```python
   from uqbinfpy import sequence
   from uqbinfpy import prob
   from uqbinfpy import seqsymbol
   from uqbinfpy import webservice
   from uqbinfpy import godata
```

### Exercise 1 (1 marks) ###

Create a “background distribution” of amino acids that is suited for scoring motifs in human protein sequences. First, consider the background used for constructing BLOSUM62 (a popular substitution matrix) by downloading and viewing `blosum62.distrib`. This file can be read using the method `readDistrib` in `prob.py`. 

```python
   bg = sequence.readDistrib('blosum62.distrib')
```

Second, construct your own from the human protein reference database (HPRD; a copy in FASTA format `HPRD.fa` is available with UniProt identifiers within the course Python code). 

Compare the backgrounds, and report if the same amino acids are the most probable and the least probable. Enter your written response into the cell labelled *Exercise 1.* This is a markdown cell. For formatting options see the [Markdown cheatsheet].

[Markdown cheatsheet]: https://github.com/adam-p/markdown-here/wiki/Markdown-Here-Cheatsheet

#### Tips: #### 
* An instance of the class `Distrib` can be inspected like so:

```python
   print bg # refers to __str__ defined for Distrib in prob.py
```

* Save the background distribution from HPRD in a file named `HPRD.distrib` using the `writeDistrib` method:

```python
   bg.writeDistrib('HPRD.distrib')
```

*Exercise 1. *

---

Helix-turn-helix (HTH) is a 20+ residue long DNA binding signature found in many transcription factors - proteins that regulate gene expression. An example structure is depicted on the right. See more at http://en.wikipedia.org/wiki/Helix-turn-helix.


## Exercise 2 ( 1 marks) ##

`hth_40.fa` contains 40 human HTH proteins. Look one up in Uniprot to see where the domain is located. Provide the accession identifier and sequence in the Markdown cell labelled *Exercise 2* and mark the HTH domain. 

**Note**: not all exemplars will be annotated with domains on Uniprot, but some will.

*Exercise 2*

---

## Exercise 3 ( 2 marks) ##

Run motif discovery by Gibbs sampling on this data a few times. The code below is a simple example of how the Gibbs sampler can be used:


```python
   from uqbinfpy import gibbs

   seqs = sequence.readFastaFile('hth_40.fa', seqsymbol.Protein_Alphabet)
   W = 24 # the width of the motif sought
   g = gibbs.GibbsMotif(seqs, W)
   q = g.discover()
   p = g.getBackground()
   a = gibbs.getAlignment(seqs, q, p)
   k = 0
   for seq in seqs:
      print "%s \t%s" % (seq.name, seq[a[k]:a[k]+W])
      k += 1
```

### Part 1)

Copy the above code and add comments explaining what the variables `seqs`, `g`, `q`, `p` and `a` represent (as an example, a comment has already been added to the variable `W`).

Enter your code into the cell labelled *Exercise 3, part 1*.


### Part 2)

Try at least three times and also with three different window sizes (`W`). Compare the outcomes of different runs. You should do this by visually comparing motifs using [WebLogo], which takes an alignment like that printed in the code example above. 

[WebLogo]: http://weblogo.berkeley.edu

Inspect and report the potential overlap between the three motifs found. Enter your written response into the markdown cell labelled *Exercise 3, part 2*. 

Select one of your motifs (the one with the highest log-odd score) and save it to a file:

```python
   prob.writeDistribs(q, 'hth_q.distrib')
   p.writeDistrib('hth_p.distrib')
```

Note that we use two files: one for the foreground and one for the background. Look in the files to understand the format (foreground has many distributions - one for each position; background has only one).


In [2]:
# Exercise 3, part 1

*Exercise 3, part 2* 

---

## Exercise 4  ( 4 marks) ##

You are now going to construct position specific scoring matrices (log-odd matrices), otherwise also called “position weight matrices” (PWMs). Construct PWMs from the probabilities defining your motif, using different definitions of background (you constructed two above, then the background in Gibbs is based on the training sequences excluding the motifs). The class `PWM` is defined in `sequence.py` module.

```python
   pwm = sequence.PWM(q, p)
```

### Part 1) ###
Use your PWMs to search for occurrences in the human protein set. Pick a few in your list of predictions, make sure they do not already occur in your “discovery set”, and look in their respective Uniprot entries. Are they likely to have a HTH domain? Enter your response into the markdown cell labelled *Exercise 4, part 1*.

### Part 2) ###

Investigate the GO terms of all your predictions as exemplified below. Note that the example only looks at the first hit in sequences with one or more matches, whereas you need to investigate all your predictions. Enter your code into the cell labelled *Exercise 4, part 2*.

```python
   ids = []
   seqs = sequence.readFastaFile('HPRD.fa', Protein_Alphabet)
   for seq in seqs:
      hits = pwm.search(seq)
      if len(hits) > 0:
         print "%s \t%d \t%s \t%5.3f" % (seq.name, hits[0][0], hits[0][1], hits[0][2])
         ids.append(seq.name)
   report = godata.getGOReport(ids) 
   for row in report:
      print "%s \t%d \t%s" % row
```

### Part 3) ###
What is the function of the discovered motif according to your analysis? Use examples from your GO term analysis to support your argument. Enter your response into the markdown cell labelled *Exercise 4, part 3*.


*Exercise 4, part 1*

In [3]:
# Exercise 4, part 2

*Exercise 4, part 3*

---